In [1]:
import requests
import pandas as pd
import os

base_socrata_url = 'http://api.us.socrata.com/api/catalog/v1'
socrata_catalog_params = {
    'min_should_match' : '1',
    'q' : ['police']
}
QUERY_LIST = ['police', 'force', 'use']
MIN_SHOULD_MATCH = '3'

In [2]:
query_str = ' '.join(QUERY_LIST)
socrata_params_str = '?min_should_match={}&q={}&limit=5000'.format(MIN_SHOULD_MATCH, query_str)
socrata_url = base_socrata_url + socrata_params_str

In [3]:
socrata_url

'http://api.us.socrata.com/api/catalog/v1?min_should_match=3&q=police force use&limit=5000'

In [4]:
socrata_resp = requests.get(socrata_url)
socrata_json = socrata_resp.json()

In [9]:
def get_resource_summary(dataset_json):
    
    dataset_resource = dataset_json['resource']
    dataset_metadata = dataset_json['metadata']
    dataset_id = dataset_resource['id']
    domain = dataset_metadata['domain']
    resource_url = os.path.join('https://', domain, 'resource', '{}.json'.format(dataset_id))
    
    resource_name = dataset_resource['name']

    dataset_summary_row = {
        'dataset_name' : resource_name,
        'dataset_url' : resource_url,
        'data_updated_at' : dataset_resource['data_updated_at'],
        'browser_url' : dataset_json['permalink'],
        'domain' : domain
    }
    
    try:
        
        resource_data = requests.get(resource_url)
        resource_df = pd.DataFrame(resource_data.json())
        resource_df = resource_df.dropna(how='all')

        resource_cols = dataset_resource['columns_field_name']
        resource_coltypes = dataset_resource['columns_datatype']
        resource_col_dict = dict(zip(resource_cols, resource_coltypes))
        
        resource_datecol_dict = {k: v for k, v in resource_col_dict.items() if 'calendar' in v.lower()}
        resource_date_cols = [col for col in resource_datecol_dict.keys()]

        resource_df = resource_df[resource_date_cols].dropna(how='any')

        most_recent_dates = []
        date_cols = []

        print(dataset_json['permalink'])
        for date_col in resource_date_cols:
            date_cols.append(date_col)
            resource_df[date_col] = pd.to_datetime(resource_df[date_col]).dt.date
            most_recent_dt = resource_df[date_col].max()
            most_recent_dates.append(most_recent_dt)

        dataset_summary_row['date_cols'] = ','.join(date_cols)
        dataset_summary_row['dataset_most_recent_dates'] = ','.join([str(dt) for dt in most_recent_dates])

    except Exception as e:
        dataset_summary_row['error'] = e
    return dataset_summary_row

In [10]:
socrata_summary_df = pd.DataFrame()

In [11]:

for idx, resource in enumerate(socrata_json['results']):
    if resource['resource']['type'] == 'dataset':
        print(idx)
        dataset_summary = get_resource_summary(resource)
        socrata_summary_df = socrata_summary_df.append(dataset_summary, ignore_index=True)

0
https://data.seattle.gov/d/ppi5-g2bj
1
https://opendata.howardcountymd.gov/d/aas5-u28t
4
https://data.nola.gov/d/9mnw-mbde
5
https://data.ny.gov/d/ca8h-8gjq
7
https://data.baltimorecity.gov/d/3w4d-kckv
8
https://data.seattle.gov/d/mg5r-efcm
9
https://data.cincinnati-oh.gov/d/8us8-wi2w
11
https://www.dallasopendata.com/d/594v-2cnd
16
https://data.seattle.gov/d/i2q9-thny
22
https://data.cityofevanston.org/d/62x8-q9xh
23
https://www.dallasopendata.com/d/99fn-pvaf
24
https://data.cityofchicago.org/d/mft5-nfa8
25
https://data.cityofchicago.org/d/ufxy-tgry
28
https://www.dallasopendata.com/d/xiv3-e8g7
32
https://data.cityofnewyork.us/d/63nx-cpi9
37
https://data.greensboro-nc.gov/d/xwnh-pcw7
39
https://data.oaklandnet.com/d/jhje-p3tp
40
https://data.cityofevanston.org/d/rayz-jh7c
41
https://data.kcmo.org/d/6wc4-sd7p
42
https://data.cityofevanston.org/d/98fk-vn7p
44
https://data.greensboro-nc.gov/d/x3i6-auhy
46
https://dashboard.edmonton.ca/d/sjhf-upbn
48
https://data.cityofevanston.org/d/sv

In [8]:
socrata_summary_df.to_csv('./police_force_use.csv', index=False)
#socrata_summary_df = pd.DataFrame()

#socrata_summary_df = socrata_summary_df.append(dataset_summary, ignore_index=True)

In [111]:
socrata_json['results'][624]

,browser_url,data_updated_at,dataset_most_recent_dates,dataset_name,dataset_url,date_cols
0,https://data.seattle.gov/d/ppi5-g2bj,2020-07-12T10:10:43.000Z,2014-10-06,Use Of Force,https://data.seattle.gov/resource/ppi5-g2bj.json,occured_date_time
1,https://opendata.howardcountymd.gov/d/aas5-u28t,2020-07-02T12:40:56.000Z,,Howard County Police Department Use Of Force ...,https://opendata.howardcountymd.gov/resource/a...,
2,https://data.nola.gov/d/9mnw-mbde,2020-02-11T21:13:56.000Z,2016-08-03,NOPD Use of Force Incidents,https://data.nola.gov/resource/9mnw-mbde.json,date_occurred
3,https://data.ny.gov/d/ca8h-8gjq,2019-10-01T22:04:56.000Z,,Index Crimes by County and Agency: Beginning 1990,https://data.ny.gov/resource/ca8h-8gjq.json,
4,https://data.baltimorecity.gov/d/3w4d-kckv,2015-12-04T16:28:46.000Z,2015-11-20,BPD Officer Involved Use Of Force,https://data.baltimorecity.gov/resource/3w4d-k...,date
...,...,...,...,...,...,...
64,https://data.cms.gov/d/din4-7td8,2015-05-06T15:46:05.000Z,,Medicare Provider Utilization and Payment Data...,https://data.cms.gov/resource/din4-7td8.json,
65,https://datahub.smcgov.org/d/kcwq-73vi,2016-04-25T15:30:41.000Z,2014-01-01,CVI Indicators Tracts,https://datahub.smcgov.org/resource/kcwq-73vi....,year
66,https://data.nola.gov/d/qtcu-97s9,2019-10-10T09:04:39.000Z,,Electronic Police Report 2017,https://data.nola.gov/resource/qtcu-97s9.json,
67,https://data.cityofnewyork.us/d/t5qg-w9sq,2018-06-07T21:52:17.000Z,,2016-2017 Student Discipline Annual Report - D...,https://data.cityofnewyork.us/resource/t5qg-w9...,


In [109]:
socrata_summary_df

,browser_url,data_updated_at,dataset_most_recent_dates,dataset_name,dataset_url,date_cols
0,https://data.seattle.gov/d/ppi5-g2bj,2020-07-12T10:10:43.000Z,2014-10-06,Use Of Force,https://data.seattle.gov/resource/ppi5-g2bj.json,occured_date_time
1,https://opendata.howardcountymd.gov/d/aas5-u28t,2020-07-02T12:40:56.000Z,,Howard County Police Department Use Of Force ...,https://opendata.howardcountymd.gov/resource/a...,
2,https://data.nola.gov/d/9mnw-mbde,2020-02-11T21:13:56.000Z,2016-08-03,NOPD Use of Force Incidents,https://data.nola.gov/resource/9mnw-mbde.json,date_occurred
3,https://data.ny.gov/d/ca8h-8gjq,2019-10-01T22:04:56.000Z,,Index Crimes by County and Agency: Beginning 1990,https://data.ny.gov/resource/ca8h-8gjq.json,
4,https://data.baltimorecity.gov/d/3w4d-kckv,2015-12-04T16:28:46.000Z,2015-11-20,BPD Officer Involved Use Of Force,https://data.baltimorecity.gov/resource/3w4d-k...,date
...,...,...,...,...,...,...
64,https://data.cms.gov/d/din4-7td8,2015-05-06T15:46:05.000Z,,Medicare Provider Utilization and Payment Data...,https://data.cms.gov/resource/din4-7td8.json,
65,https://datahub.smcgov.org/d/kcwq-73vi,2016-04-25T15:30:41.000Z,2014-01-01,CVI Indicators Tracts,https://datahub.smcgov.org/resource/kcwq-73vi....,year
66,https://data.nola.gov/d/qtcu-97s9,2019-10-10T09:04:39.000Z,,Electronic Police Report 2017,https://data.nola.gov/resource/qtcu-97s9.json,
67,https://data.cityofnewyork.us/d/t5qg-w9sq,2018-06-07T21:52:17.000Z,,2016-2017 Student Discipline Annual Report - D...,https://data.cityofnewyork.us/resource/t5qg-w9...,


In [92]:
dataset = socrata_json['results'][78]

dataset_resource = dataset['resource']
dataset_metadata = dataset['metadata']

dataset_id = dataset_resource['id']
domain = dataset_metadata['domain']


In [104]:
resource_url = os.path.join('https://', domain, 'resource', '{}.json?$limit=100000000'.format(dataset_id))

resource_url

'https://data.nola.gov/resource/jsyu-nz5r.json?$limit=100000000'

In [105]:
resource_data = requests.get(resource_url)
resource_df = pd.DataFrame(resource_data.json())
resource_df['timearrive'].isna().sum()

96166

In [106]:
resource_df.shape

(447462, 29)

In [98]:
resource_name = dataset_resource['name']
resource_cols = dataset_resource['columns_field_name']
resource_coltypes = dataset_resource['columns_datatype']
resource_col_dict = dict(zip(resource_cols, resource_coltypes))
resource_datecol_dict = {k: v for k, v in resource_col_dict.items() if 'calendar' in v.lower()}
resource_date_cols = [col for col in resource_datecol_dict.keys()]

In [101]:
resource_df.shape

(1000, 29)

In [84]:
dataset_summary_row = {
    'dataset_name' : resource_name,
    'dataset_url' : resource_url,
    'data_updated_at' : dataset_resource['data_updated_at']
}
most_recent_dates = []
date_cols = []

for date_col in resource_date_cols:
    date_cols.append(date_col)
    resource_df[date_col] = pd.to_datetime(resource_df[date_col]).dt.date
    most_recent_dt = resource_df[date_col].max()
    most_recent_dates.append(most_recent_dt)
    
dataset_summary_row['date_cols'] = ','.join(date_cols)
dataset_summary_row['dataset_most_recent_dates'] = ','.join([str(dt) for dt in most_recent_dates])
#resource_df[resource_date_cols] = pd.to_datetime(resource_df[resource_date_cols], format='%Y-%m-%dT%H:%M:%S.%f')

TypeError: '>=' not supported between instances of 'datetime.date' and 'float'

In [97]:
resource_date_cols

['year']

In [56]:
#dt_formats = ['%Y-%m-%dT%H:%M:%S.%f']

resource_df[date_col].max()

datetime.date(2014, 10, 6)

In [55]:
dataset_summary_row

{'dataset_name': 'Use Of Force',
 'dataset_url': 'https://data.seattle.gov/resource/ppi5-g2bj.json',
 'data_updated_at': '2020-07-12T10:10:43.000Z',
 'date_cols': 'occured_date_time',
 'dataset_most_recent_dates': '2014-10-06'}